<a href="https://colab.research.google.com/github/pmachadors/Recomendador-Spotify/blob/main/recomendador_spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 236 kB 6.9 MB/s 
     |████████████████████████████████| 140 kB 42.7 MB/s 
     |████████████████████████████████| 62 kB 512 kB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 199 kB 50.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=87ac724637a97c137994f8a0cdd587ea31e17153a019d04bb54b7915827b007d
  Stored in directory: /root/.cac

# Importando Bibliotecas

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType
from pyspark import SparkFiles

# Leitura da base

In [ ]:
sessao_spark = SparkSession.builder.appName('recomendador_spotify').getOrCreate()

url_dados = 'https://github.com/IgorNascAlves/dados/blob/main/dados_musicas.csv?raw=true'
sessao_spark.sparkContext.addFile(url_dados)
path_dados_file = 'file://' + SparkFiles.get('dados_musicas.csv')
dados = sessao_spark.read.csv(path_dados_file, header=True, sep=';', inferSchema=True)

dados.show(5, truncate = False)

+------------------+----+------------+------------+------------+-----------+------------------+--------+----------------------+----------------+---+--------+-------------------+----+-------------------+----------+-----------+-------+----------------------------+
|valence           |year|acousticness|artists     |danceability|duration_ms|energy            |explicit|id                    |instrumentalness|key|liveness|loudness           |mode|name               |popularity|speechiness|tempo  |artists_song                |
+------------------+----+------------+------------+------------+-----------+------------------+--------+----------------------+----------------+---+--------+-------------------+----+-------------------+----------+-----------+-------+----------------------------+
|0.285             |2000|0.00239     |Coldplay    |0.429       |266773     |0.6609999999999999|0       |3AJwUDP919kvQ9QcozQPxg|1.21E-4         |11 |0.234   |-7.227             |1   |Yellow             |84       

# Criando sessão Spotipy

In [ ]:
scope = "user-library-read playlist-modify-private"

OAuth = SpotifyOAuth(
        scope=scope,         
        redirect_uri='http://localhost:5000/callback',
        client_id = '9c72f0a29993448c84b3b76265f4393a',
        client_secret = '94d2411b2f094ea187dcdf27a45043f4')


In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id = '9c72f0a29993448c84b3b76265f4393a',
                                                      client_secret = '94d2411b2f094ea187dcdf27a45043f4')

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# Pipeline

In [ ]:
x = dados.columns
x.remove('artists')
x.remove('id')
x.remove('name')
x.remove('artists_song')
x


['valence',
 'year',
 'acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'explicit',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'popularity',
 'speechiness',
 'tempo']

In [ ]:
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=x,outputCol='features'),\
                StandardScaler(inputCol='features', outputCol='scaled_features'),\
                PCA(k=6, inputCol='scaled_features',outputCol='pca_features')\
                ])

model_pipeline = pca_pipeline.fit(dados)
projection = model_pipeline.transform(dados)

projection.select('pca_features').show(5, truncate=False)


+----------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                          |
+----------------------------------------------------------------------------------------------------------------------+
|[-34.71004464775704,-165.36848784906178,-11.163498729833321,-139.11374280854434,-14.152192300931375,6.254422223810391]|
|[-36.49885762774905,-167.85185045616555,-10.729023248907561,-135.904352656713,-13.388401890587229,3.9729807278308606] |
|[-35.460300916940554,-165.99633344577884,-11.361290348241592,-138.24276881247,-13.866654919305779,4.342675920458719]  |
|[-35.56301325520584,-165.59515096480607,-11.460226007031382,-137.52095740570041,-15.56854280392272,4.542073725584271] |
|[-36.54556974907567,-167.37512505802482,-11.881276527236555,-136.27798590243984,-14.05684799034454,3.583390145779156] |
+-------------------------------

# K-means

In [ ]:
SEED = 1 

kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca',k=50, seed=SEED)
model_kmeans = kmeans.fit(projection)

projection_kmeans = model_kmeans.transform(projection)

projection_kmeans.select('pca_features','cluster_pca').show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------+-----------+
|pca_features                                                                                                          |cluster_pca|
+----------------------------------------------------------------------------------------------------------------------+-----------+
|[-34.71004464775704,-165.36848784906178,-11.163498729833321,-139.11374280854434,-14.152192300931375,6.254422223810391]|46         |
|[-36.49885762774905,-167.85185045616555,-10.729023248907561,-135.904352656713,-13.388401890587229,3.9729807278308606] |16         |
|[-35.460300916940554,-165.99633344577884,-11.361290348241592,-138.24276881247,-13.866654919305779,4.342675920458719]  |14         |
|[-35.56301325520584,-165.59515096480607,-11.460226007031382,-137.52095740570041,-15.56854280392272,4.542073725584271] |41         |
|[-36.54556974907567,-167.37512505802482,-11.881276527236555,-136.277

# Recomendador de músicas

In [ ]:
def recomendador(nome_musica):
  # Calcula musicas recomendadas
  cluster = projection_kmeans.filter(projection_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
  musicas_recomendadas = projection_kmeans.filter(projection_kmeans.cluster_pca == cluster)\
                                       .select('artists_song', 'id', 'pca_features')
  componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                          .select('pca_features').collect()[0][0]

  def calcula_distance(value):
    return euclidean(componenetes_musica, value)

  udf_calcula_distance = f.udf(calcula_distance, FloatType())

  musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

  recomendadas = sessao_spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

  recomendadas.select('id','artists_song').show(truncate=False)

  # Pegar informações da API 

  playlist_id = recomendadas.select('id').collect()

  playlist_track = []

  for id in playlist_id:
    playlist_track.append(sp.track(id[0]))

  return len(playlist_track)

In [ ]:
recomendador('Linkin Park - In the End')
# track_id = sp.search(q='artist:' + artist + ' track:' + track, type='track')
# track_id


+----------------------+--------------------------------------+
|id                    |artists_song                          |
+----------------------+--------------------------------------+
|60a0Rd6pjrkxjPbaKzXjfq|Linkin Park - In the End              |
|4vqmSmLC0dbw1UF7mmUDZe|Falling In Reverse - Raised By Wolves |
|5jnp8E61wcoF6qIEtDHQnJ|ODESZA - All We Need (feat. Shy Girls)|
|4Sfa7hdVkqlM8UW5LsSY3F|Passion Pit - Take a Walk             |
|4E6cwWJWZw2zWf7VFbH7wf|Sara Bareilles - Love Song            |
|5r00wpsojd4mBYaQFDvni7|Luke Bryan - Roller Coaster           |
|2aMb1asq5acm7cDYlFsYhY|Chris Tomlin - Home                   |
|3pOMQhSSzx6IZ96kMef6i1|Pop Evil - Be Legendary               |
|2Kl1E3NdbzT7vVi93TffoX|Bastille - Pompeii - Audien Remix     |
|7I3skNaQdvZSS7zXY2VHId|Twenty One Pilots - The Hype          |
+----------------------+--------------------------------------+



10